In [19]:
%load_ext autoreload
%autoreload 2
import math
from functools import partial
import uproot
import numpy as np
import pandas as pd
import awkward as ak
from tqdm import tqdm
import hist

import sys
sys.path.append("../../..")
from hists.parameters import thresholdW0 as defaultThresholdW0
from hists.parameters import synchrotronBeamEnergiesMap
from hists.dataframe import DataframeComputations
from HistogramLib.store import HistogramStore
from hists.store import HistogramId

from event_visualizer.plotter.layer import LayerVisualization
from event_visualizer.plotter.clue3D import Clue3DVisualization
from event_visualizer.event_index import EventLoader, EventID, LoadedEvent
from locateEvents.utils import makeDashLink, makeCsvRow, printCsvRowsFromDf
from locateEvents.sample_chooser import EventDisplayList

import locateEvents.displacedLayerCluster.dataframe_making as dlc_find
from locateEvents.displacedLayerCluster.dataframe_making import findDisplacedLC, barycenterDifferenceFromDefault, makeFullBarycenterDf

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
input_file = '/data_cms_upgrade/cuisset/testbeam18/clue3d/v31/cmssw/data/CLUE_clusters.root'
tree = uproot.open(input_file + ":clusters")
eventLoader = EventLoader(input_file)

In [12]:
array = tree.arrays(entry_stop=1000)
comp_test = DataframeComputations(array, ["rechits_x", "rechits_y", "rechits_layer", "rechits_energy"])  #.set_index(["event", "clus3D_id", "clus2D_id"])

In [28]:
comp_test.clusters3D_with_clus2D_id()

clus3D_energy  beamEnergy  clus3D_size  clus2D_id
eventInternal clus3D_id                                                   
0             0             144.070526       300.0           33          0
              0             144.070526       300.0           33          2
              0             144.070526       300.0           33          4
              0             144.070526       300.0           33          6
              0             144.070526       300.0           33          9
...                                ...         ...          ...        ...
999           0             279.443359       300.0           35         61
              0             279.443359       300.0           35         62
              0             279.443359       300.0           35         63
              0             279.443359       300.0           35         64
              0             279.443359       300.0           35         65

[39980 rows x 4 columns]

In [32]:
bary_df = makeFullBarycenterDf(comp_test)
bary_df

,beamEnergy_x,clus2D_x,clus2D_y,clus2D_layer,clus2D_energy,clus2D_size,clus2D_x_barycenter,clus2D_y_barycenter,clus2D_distance_positionToBarycenter,rechits_energy_sum_perLayer,impactX,impactY,clus2D_distance_positionToImpact,clus2D_distance_barycenterToImpact,clus3D_id,clus3D_energy,beamEnergy_y,clus3D_size,clus2D_id
eventInternal,,,,,,,,,,,,,,,,,,,
0,300.0,2.471932,-1.914578,1,3.747497,25,2.109860,-1.825651,3.728326e-01,4.312928,2.444633,-0.835710,1.079214,1.045015,0.0,144.070526,300.0,33.0,0
0,300.0,0.703080,3.054918,1,0.275047,3,0.703080,3.054918,9.805389e-08,4.312928,2.444633,-0.835710,4.262628,4.262628,NaN,NaN,NaN,NaN,1
0,300.0,2.520951,-1.829756,2,5.297332,33,2.520951,-1.829756,2.817517e-07,5.497473,2.445000,-0.836198,0.996457,0.996457,0.0,144.070526,300.0,33.0,2
0,300.0,5.625668,0.263639,2,0.163312,2,5.625668,0.263639,1.346721e-08,5.497473,2.445000,-0.836198,3.365455,3.365455,NaN,NaN,NaN,NaN,3
0,300.0,2.364095,-1.885765,3,10.562175,44,2.364095,-1.885765,1.785139e-07,10.904448,2.445830,-0.837301,1.051645,1.051645,0.0,144.070526,300.0,33.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999,300.0,2.153780,-1.696138,26,0.132718,4,2.726776,-1.965416,6.331164e-01,0.227178,1.747391,-1.646739,0.409380,1.029927,0.0,279.443359,300.0,35.0,61
999,300.0,4.603579,-0.658863,26,0.020060,2,4.603578,-0.658863,6.337388e-07,0.227178,1.747391,-1.646739,3.022202,3.022202,0.0,279.443359,300.0,35.0,62
999,300.0,2.938763,-1.082473,27,0.175747,5,2.335440,-1.388820,6.766434e-01,0.239784,1.748215,-1.647772,1.317940,0.641786,0.0,279.443359,300.0,35.0,63


In [15]:
new_res = findDisplacedLC(tree, [lookupFractionEnergyOnLayer], maxCount=10)

  0%|          | 0/281873 [00:13<?, ?it/s]


In [17]:
new_res["lookupFractionEnergyOnLayer"]

,beamEnergy,clus2D_x,clus2D_y,clus2D_layer,clus2D_energy,clus2D_size,clus2D_x_barycenter,clus2D_y_barycenter,clus2D_distance_positionToBarycenter,rechits_energy_sum_perLayer,impactX,impactY,clus2D_distance_positionToImpact,clus2D_distance_barycenterToImpact,ntupleNumber,event,beamEnergy_r
eventInternal,,,,,,,,,,,,,,,,,
43,300.0,0.083081,0.781868,24,0.650591,18,1.953546,-0.309308,2.165480,0.771582,1.792296,-0.245380,1.994156,0.173459,435,1442,300.0
44,300.0,4.000311,-3.610464,28,0.249591,12,2.343994,-2.463412,2.014724,0.342563,2.134623,-1.780288,2.613492,0.714490,435,1444,300.0
64,300.0,0.094261,-0.355702,21,1.570499,25,1.969218,-1.896633,2.426919,1.735797,2.106365,-1.757184,2.452084,0.195591,435,1509,300.0
111,300.0,-0.895730,0.283688,23,1.328202,24,1.083024,-0.494881,2.126415,1.536511,1.810813,-0.365353,2.783277,0.739225,435,1632,300.0
134,300.0,4.955517,0.028265,25,1.098429,18,2.895848,-0.594686,2.151814,1.368850,2.418825,-0.104225,2.540150,0.684181,435,1723,300.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8969,20.0,0.927120,-3.181182,11,1.418226,16,2.007993,-1.324262,2.148590,1.448841,2.282433,-0.443164,3.055097,0.922849,441,6367,20.0
9111,20.0,0.927120,0.194428,13,0.361376,12,2.530043,-1.181587,2.112529,0.459460,3.124226,-1.173263,2.588021,0.594242,441,7251,20.0
9230,20.0,2.935278,-3.223448,12,0.422722,13,2.756200,-1.220201,2.011235,0.474651,2.087733,-1.583206,1.846274,0.760672,441,10244,20.0
